see https://help.marine.copernicus.eu/en/articles/6725319-how-to-download-data-for-a-recurring-period-over-several-years-via-the-motu-client-in-python 
for howto make this cli command into a program

In [ ]:
# Base command example
# python -m motuclient --motu https://my.cmems-du.eu/motu-web/Motu
# --service-id WIND_GLO_PHY_L4_MY_012_006-TDS
# --product-id cmems_obs-wind_glo_phy_my_l4_0.125deg_PT1H
# --longitude-min -12.455343303571427 --longitude-max 16.473195982142855
# --latitude-min 45.241023613016935 --latitude-max 62.91957539873122
# --date-min "2021-01-01 00:00:00" --date-max "2021-01-31 12:00:00"
# --variable eastward_wind --variable northward_wind
# --out-dir ./ --out-name test.nc --user fborg --pwd Zx7iwpWtqKqPTCG

In [ ]:
import os
import subprocess
from datetime import datetime, timedelta
import getpass
import xarray as xr
import math
from tqdm import tqdm

In [ ]:
#Backup settings
#out_dir = '/data/exjobb/sarssw/model/WIND_GLO_PHY'
#lon = (-12 , 16.5)
#lat = (45 , 63)
#This also used the old intervals
#sec = timedelta(seconds=1)
#start_dates = [datetime(2021, i, 1, 0) for i in range(1, 13)]
#end_dates = [d - sec for d in start_dates[1:]] + [datetime(2021, 12, 31, 23)]
#date_intervals = list(zip(start_dates, end_dates))

In [ ]:
# Copernicus Marine Credentials 
USERNAME = input('Enter your username: ')
PASSWORD = getpass.getpass("Enter your password: ")

In [ ]:
# Work directory
out_dir = '/data/exjobb/sarssw/model/WIND_GLO_PHY_global'

#Conditionally creates the folder for the result
os.makedirs(out_dir, exist_ok=True)

#Check if we alreay have files and raise exception if we dont know how to continue
if os.path.exists(out_dir) and os.path.isdir(out_dir):
    files = os.listdir(out_dir)
    #Raise if we find other files than number.nc
    if not all([file[:-3].isnumeric() for file in files]):
        raise Exception(f"dir {out_dir} contains unexpected files, the files {os.listdir(out_dir)} exist")

files = os.listdir(out_dir)

# product and dataset IDs
# For https://data.marine.copernicus.eu/product/WIND_GLO_PHY_L4_MY_012_006/description
serviceID = "WIND_GLO_PHY_L4_MY_012_006"
productID = "cmems_obs-wind_glo_phy_my_l4_0.125deg_PT1H"

#coordinates
lon = (-180 , 180)
lat = (-80, 80)

# variable 
vars = ["eastward_wind", "northward_wind"]

# Boundary dates
time_step = timedelta(days=6)
sec = timedelta(seconds=1)
start_date = datetime(2021, 1, 1, 0, 0)
end_date = datetime(2021, 12, 31, 23, 59)
n_steps = math.ceil((end_date-start_date)/time_step)

start_dates = [start_date + i*time_step for i in range(n_steps)]
end_dates = [start_date + i*time_step - sec for i in range(1,n_steps)] + [end_date]
date_intervals = list(zip(start_dates, end_dates))

all_files_req = [str(i)+'.nc' for i in range(len(date_intervals))]

In [ ]:
#If download failed last time then adjust to only donwload the files that failed last time
file_date_zip = [tup for tup in zip(all_files_req, date_intervals) if tup[0] not in files]
print("Files to download:\n", file_date_zip)

# Download loop 
for out_name, (start_date, end_date) in tqdm(file_date_zip):
    
    # Motuclient command line
    cmd = f'python -m motuclient --motu https://my.cmems-du.eu/motu-web/Motu \
    --service-id {serviceID}-TDS --product-id {productID} \
    --longitude-min {lon[0]} --longitude-max {lon[1]} --latitude-min {lat[0]} --latitude-max {lat[1]}\
    --date-min "{start_date}" --date-max "{end_date}" \
    {" ".join([f"--variable {v}" for v in vars])} \
    --out-dir {out_dir} --out-name {out_name} --user {USERNAME} --pwd {PASSWORD}'        

    print(f"============== Running request on {start_date} ==============")
    #print(cmd)
    
    # Run the command
    if os.system(cmd) != 0:
        raise Exception('Error')

        
print(f"============== Download completed! All files are in your directory {out_dir} ==============")

In [ ]:
#Check that all files have been downloaded
file_list = os.listdir(out_dir)
if (not all(file in file_list for file in all_files_req)) or (not all(file in all_files_req for file in file_list)):
   raise Exception('Not all necessary files was downloaded, rerun the previous cell')

In [ ]:
#Test load one dataset
cop_wind_ds = xr.open_dataset(os.path.join(out_dir, file_list[-1]))
cop_wind_ds

In [ ]:
#Check that all scale_factors and add_offsets are the same, if not rase Exception
def scale_offset_attr(dir, file):
    cmd_ncks = f"ncks -m -M {os.path.join(dir, file)}".split(' ')
    #cmd_ncks = 'ls'
    cmd_grep = "grep -E scale_factor|add_offset".split(' ')
    #cmd_grep = "wc".split(' ')
    pipe = subprocess.Popen(cmd_ncks, stdout=subprocess.PIPE)
    try:
        res = subprocess.check_output(cmd_grep, stdin=pipe.stdout).decode('ascii')
    #If grep finds nothing return empty dict
    except subprocess.CalledProcessError:
        return ()

    return tuple(sorted(res.replace(' ', '').replace('\n', '').split(';')))

unique_scale_offset = set([scale_offset_attr(out_dir, file) for file in all_files_req])

#If we have 0 or 1 unique values of scale_offsets it is okay, anything else means we have different scale or offset
if len(unique_scale_offset) > 1:
    raise Exception(f'Scale and/or offset is not the same for all files\n{unique_scale_offset}')

In [ ]:
#Add record variable (time) to all files
int_file_list_sorted = sorted([int(f[:-3]) for f in file_list if f[-3:] == '.nc'])
for file_int in tqdm(int_file_list_sorted):
    cmd = f"ncks --mk_rec_dmn time {os.path.join(out_dir, str(file_int))}.nc -o {os.path.join(out_dir, str(file_int))}_rec.nc"
    #print(cmd)
    # Run the command
    if os.system(cmd) != 0:
        raise Exception('Error')

In [ ]:
#Join the datasets
save_file = 'all.nc'
cmd = f"ncrcat {' '.join([os.path.join(out_dir, str(file_int)) + '_rec.nc' for file_int in int_file_list_sorted])} {os.path.join(out_dir,save_file)}"
#print(cmd)
# Run the command
if os.system(cmd) != 0:
        raise Exception('Error')

In [ ]:
#Show the final dataset
ds = xr.open_dataset(os.path.join(out_dir, save_file))
display(ds)

In [ ]:
#Clean up the direcory
rm_files = os.listdir(out_dir)
rm_files.remove(save_file)
rm_files
for file in tqdm(rm_files):
    cmd = f"rm {os.path.join(out_dir, file)}"
    if os.system(cmd) != 0:
        raise Exception('Error')